In [ ]:
def format_time(start_time, init_time):
    # Calculate the difference in time
    diff = start_time - init_time

    # Get the total number of seconds
    total_seconds = diff.total_seconds()

    # Calculate hours, minutes, and seconds
    hours, remainder = divmod(total_seconds, 3600)
    minutes, seconds = divmod(remainder, 60)

    # Calculate milliseconds
    milliseconds = int((seconds % 1) * 1000)

    # Format the time
    time_str = "{:02}:{:02}:{:02},{:03}".format(int(hours), int(minutes), int(seconds), milliseconds)

    return time_str

In [ ]:
def cut_sentence(sentence, max_chars):
    # Split the sentence into individual Chinese characters
    chars = list(sentence)
    
    # Initialize variables
    sentences = []
    current_sentence = ""
    current_chars = 0
    
    # Iterate through each character
    for char in chars:
        # Check if adding the current character exceeds the maximum character limit
        if current_chars + len(char) > max_chars:
            # Add the current sentence to the list of sentences
            sentences.append(current_sentence)
            
            # Reset the current sentence and character count
            current_sentence = ""
            current_chars = 0
        
        # Add the current character to the current sentence
        current_sentence += char
        current_chars += len(char)
    
    # Add the last sentence to the list of sentences
    sentences.append(current_sentence)
    
    return sentences

In [ ]:
def get_srt_file(ref_wav_path, prompt_text, prompt_language, text, text_language,srt_file_path,srt_gen=True,
                 how_to_cut=i18n("按标点符号切"), top_k=20, top_p=0.6, temperature=0.6, ref_free = False):
    if prompt_text is None or len(prompt_text) == 0:
        ref_free = True
    t0 = ttime()
    prompt_language = dict_language[prompt_language]
    text_language = dict_language[text_language]
    if not ref_free:
        prompt_text = prompt_text.strip("\n")
        if (prompt_text[-1] not in splits): prompt_text += "。" if prompt_language != "en" else "."
        print(i18n("实际输入的参考文本:"), prompt_text)
    text = text.strip("\n")
    if (text[0] not in splits and len(get_first(text)) < 4): text = "。" + text if text_language != "en" else "." + text
    
    print(i18n("实际输入的目标文本:"), text)
    zero_wav = np.zeros(
        int(hps.data.sampling_rate * 0.3),
        dtype=np.float16 if is_half == True else np.float32,
    )
    with torch.no_grad():
        wav16k, sr = librosa.load(ref_wav_path, sr=16000)
        if (wav16k.shape[0] > 160000 or wav16k.shape[0] < 48000):
            raise OSError(i18n("参考音频在3~10秒范围外，请更换！"))
        wav16k = torch.from_numpy(wav16k)
        zero_wav_torch = torch.from_numpy(zero_wav)
        if is_half == True:
            wav16k = wav16k.half().to(device)
            zero_wav_torch = zero_wav_torch.half().to(device)
        else:
            wav16k = wav16k.to(device)
            zero_wav_torch = zero_wav_torch.to(device)
        wav16k = torch.cat([wav16k, zero_wav_torch])
        ssl_content = ssl_model.model(wav16k.unsqueeze(0))[
            "last_hidden_state"
        ].transpose(
            1, 2
        )  # .float()
        codes = vq_model.extract_latent(ssl_content)
   
        prompt_semantic = codes[0, 0]
    t1 = ttime()

    if (how_to_cut == i18n("凑四句一切")):
        text = cut1(text)
    elif (how_to_cut == i18n("凑50字一切")):
        text = cut2(text)
    elif (how_to_cut == i18n("按中文句号。切")):
        text = cut3(text)
    elif (how_to_cut == i18n("按英文句号.切")):
        text = cut4(text)
    elif (how_to_cut == i18n("按标点符号切")):
        text = cut5(text)
    while "\n\n" in text:
        text = text.replace("\n\n", "\n")
    print(i18n("实际输入的目标文本(切句后):"), text)
    texts = text.split("\n")
    texts = merge_short_text_in_array(texts, 5)
    audio_opt = []
    if not ref_free:
        phones1,bert1,norm_text1=get_phones_and_bert(prompt_text, prompt_language)
    srt_entries = []
    
    for text in texts:
        # 解决输入目标文本的空行导致报错的问题
        if (len(text.strip()) == 0):
            continue
        if (text[-1] not in splits): text += "。" if text_language != "en" else "."
        print(i18n("实际输入的目标文本(每句):"), text)
        phones2,bert2,norm_text2=get_phones_and_bert(text, text_language)
        print(i18n("前端处理后的文本(每句):"), norm_text2)
        if not ref_free:
            bert = torch.cat([bert1, bert2], 1)
            all_phoneme_ids = torch.LongTensor(phones1+phones2).to(device).unsqueeze(0)
        else:
            bert = bert2
            all_phoneme_ids = torch.LongTensor(phones2).to(device).unsqueeze(0)

        bert = bert.to(device).unsqueeze(0)
        all_phoneme_len = torch.tensor([all_phoneme_ids.shape[-1]]).to(device)
        prompt = prompt_semantic.unsqueeze(0).to(device)
        t2 = ttime()
        with torch.no_grad():
            # pred_semantic = t2s_model.model.infer(
            pred_semantic, idx = t2s_model.model.infer_panel(
                all_phoneme_ids,
                all_phoneme_len,
                None if ref_free else prompt,
                bert,
                # prompt_phone_len=ph_offset,
                top_k=top_k,
                top_p=top_p,
                temperature=temperature,
                early_stop_num=hz * max_sec,
            )
        t3 = ttime()
        # print(pred_semantic.shape,idx)
        pred_semantic = pred_semantic[:, -idx:].unsqueeze(
            0
        )  # .unsqueeze(0)#mq要多unsqueeze一次
        refer = get_spepc(hps, ref_wav_path)  # .to(device)
        if is_half == True:
            refer = refer.half().to(device)
        else:
            refer = refer.to(device)
        # audio = vq_model.decode(pred_semantic, all_phoneme_ids, refer).detach().cpu().numpy()[0, 0]
        audio = (
            vq_model.decode(
                pred_semantic, torch.LongTensor(phones2).to(device).unsqueeze(0), refer
            )
                .detach()
                .cpu()
                .numpy()[0, 0]
        )  ###试试重建不带上prompt部分
        ### ——————————————生成字幕————————————————————————
        audio_duration = len(audio) / hps.data.sampling_rate
        text = text.replace('。','').replace(',','')
        if len(srt_entries) == 0:
            init_time = datetime.datetime.now()
            start_time = init_time+datetime.timedelta(seconds=0)
        cut_sub_list = cut_sentence(text,8)
        if len(cut_sub_list)>1:
            audio_piece = audio_duration/len(cut_sub_list)
            for sub in cut_sub_list:
                end_time = start_time+datetime.timedelta(seconds=audio_piece)
                srt_entry = f"{len(srt_entries) + 1}\n{format_time(start_time,init_time)} --> {format_time(end_time,init_time)}\n{sub}\n\n"
                start_time = end_time
                #print(srt_entry)
                srt_entries.append(srt_entry)
        else:
            end_time = start_time+datetime.timedelta(seconds=audio_duration)
            srt_entry = f"{len(srt_entries) + 1}\n{format_time(start_time,init_time)} --> {format_time(end_time,init_time)}\n{text}\n\n"
            start_time = end_time
            #print(srt_entry)
            srt_entries.append(srt_entry)
        ### ————————————————生成字幕结束——————————————————————
        t4 = ttime()
    print("%.3f\t%.3f\t%.3f\t%.3f" % (t1 - t0, t2 - t1, t3 - t2, t4 - t3))
    if srt_gen:
        with open(srt_file_path, "w",encoding='utf-8') as srt_file:
            srt_file.writelines(srt_entries)
            print('srt generated')



In [ ]:
import subprocess,time
st=time.time()
proc = subprocess.Popen(['ffmpeg','-y','-i','xuniren/final/保单分析帮你分析保险回本能力.mp4','-vf','subtitles=temp/subtitle.srt','temp/sub_video.mp4'])
proc.wait()
print(f'run time:{time.time()-st}')

In [ ]:
: TextClip(txt, font='Alibaba-PuHuiTi-2-85-Bold', font_size=75, color='white',method='caption',align='center',size=(900,1920),kerning=3,stroke_color='black',stroke_width=2)